<a href="https://colab.research.google.com/github/JavierPalmero23/Bird-Identification-and-Damage-Control-with-Jetson-Nano/blob/main/PyTorch/Model-Build-And-Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library and System

It is necessary to import the appropriate libraries and prepare everything.

For that, we'll update the Python Libraries.

In [ ]:
!pip install --upgrade torch torchvision

It is always recommended to clear the cache to avoid filling the storage and loading corrupted data


In [14]:
!rm -rf ~/.cache/*

In [4]:
import os

import numpy as np
from tqdm import tqdm
from time import time
from PIL import Image

import torch
from torch.optim import Adam
from torch.nn import CrossEntropyLoss, Conv2d, AvgPool2d, BatchNorm2d, Dropout2d, LeakyReLU, Linear, Module
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from torchvision.transforms import transforms

# Checks if GPU us available with cuda drivers
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Storage Load

For **LOCAL** Storage

In [ ]:
!pip install gitpython

import git

git.Git("/content").clone("https://github.com/JavierPalmero23/Bird-Identification-and-Damage-Control-with-Jetson-Nano.git")

path = "/content/Bird-Identification-and-Damage-Control-with-Jetson-Nano/PyTorch/Birds"


In case you're using **GOOGLE DRIVE** use the next line.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab DataSets/BIRD/"

The Data Transformer allows us to keep a standard format for the pictures

In [7]:
# Data Transformer
tfm=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])


Now, we'll define the path to the training and test pictures. Also, define the number of pictures to keep for later processing


In [11]:
# Create Dataset
TRAIN_ROOT = path+'/train'
TEST_ROOT = path+'/test'

# Length of Train and Test Datasets
train_ds = ImageFolder(TRAIN_ROOT, transform=tfm)
test_ds = ImageFolder(TEST_ROOT, transform=tfm)

# Length of Train and Test Datasets
len_train = len(train_ds)
len_test = len(test_ds)


In [ ]:
# Index Mapping
train_ds.class_to_idx


In [17]:
# Data Loader
train_loader = DataLoader(train_ds, batch_size=30, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=30, shuffle=True)


# Build the Model

In [18]:
# Build Model
class ModBuilder(Module):
    def __init__(self):
        super(ModBuilder, self).__init__()
        self.conv = Conv2d(in_channels=3, out_channels=8, kernel_size=(3,3), stride=1, padding=1)
        self.pool = AvgPool2d(kernel_size=(3,3), stride=1)
        self.relu = LeakyReLU()
        self.bn = BatchNorm2d(num_features=8)
        self.drop = Dropout2d(p=0.3)
        self.fc = Linear(in_features=8*126*126, out_features=16)

    def forward(self, X):
        output = self.conv(X)
        output = self.pool(output)
        output = self.relu(output)
        output = self.bn(output)
        output = self.drop(output)
        print(output.shape)
        output = output.view(-1, 8*126*126)
        output = self.fc(output)
        return output


In case you want to ***LOAD*** a ***Previous Model***, you can load it with the next line and ***SKIP*** the ***Model Definition***

In [ ]:
# Model Load
model = torch.load(path+'/model.pt')
model = model.to(device)


In [14]:
# Model Definition and Migration to GPU
model = ModBuilder().to(device)


A few modifications will be good to optimize the model performance

In [15]:
# Optimiser
optimiser = Adam(model.parameters(), lr=3e-4, weight_decay=0.0001)

# Loss Function
loss_fn = CrossEntropyLoss()


# Train the Model

Keep in mind that the model not always will have the same efficiency with 10 or 15 rounds of training, so feel free to use more rounds

> *Keep in mind that the first round or epoch will take more time than the other*


In [ ]:
for epoch in range(10):
    start = time()
    train_acc = 0
    test_acc = 0

# Training
    model.train()
    with tqdm(train_loader, unit="batch") as tepoch:
        for xtrain, ytrain in tepoch:
            optimiser.zero_grad()
            xtrain = xtrain.to(device)
            train_prob = model(xtrain)
            train_prob = train_prob.cpu()
            train_loss = loss_fn(train_prob, ytrain)
            train_loss.backward()
            optimiser.step()
            # END TRAINING
            train_pred = torch.max(train_prob, 1).indices
            train_acc += int(torch.sum(train_pred == ytrain))

        train_epoch_accuracy = train_acc / len_train

# Evaluate
    model.eval()
    with torch.no_grad():
        for xtest, ytest in test_loader:
            xtest = xtest.to(device)
            test_prob = model(xtest)
            test_prob = test_prob.cpu()
            test_loss = loss_fn(test_prob, ytest)
            test_pred = torch.max(test_prob, 1).indices
            test_acc += int(torch.sum(test_pred == ytest))

        test_epoch_accuracy = test_acc / len_test

    end = time()

    diff = end - start

    print(f"Epoch: {epoch+1}, Time: {diff}\nTr_loss: {train_loss}, Test_loss: {test_loss}\n,Tr_acc:{train_epoch_accuracy}, Test_acc: {test_epoch_accuracy}")


# Test the Model

For this example we'll be testing the model with a few pictures.

In [14]:
# Image Samples
sample_1 = path+'/test/woodpecker/ERDE66G1GTCV.jpg'
sample_2 = path+'/test/duck/F5A2MH75RPWP.jpg'
sample_3 = path+'/test/common_grackle/O91IJQ5JYMCH.jpg'
sample_4 = path+'/test/crow/93FKTKYFIME8.jpg'

sample_list = [sample_1,sample_2,sample_3,sample_4]


Now it's time to run the test and check the Model Performance

In [ ]:
# Model Performance
for img_path in sample_list:
    img = Image.open(img_path)
    display(img.resize((100, 100)))
    img_tensor = tfm(img)
    img_tensor = img_tensor[np.newaxis, :]
    img_tensor = img_tensor.to(device)
    prob = model(img_tensor)
    pred = torch.max(prob, 1).indices
    pred = pred.item()
    print(f"Model Prediction: {pred}")
    print(" ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~")


In case that the model isn't accurate enough, a retraining will be good

# Save the Model

If the model have a good performance or you want to keep with the training later, you can save it with the next line of code

In [21]:
# Save the Model
torch.save(model, path+'/model.pt')
